In [11]:
# Import libraries
import mailbox
import bs4
import email
from email import message
import quopri,base64
import pandas as pd
import sys
sys.path.append('..')

In [12]:
mbox_data = mailbox.mbox('../Data/Destacados.mbox')
num_emails = len(mbox_data)
num_emails
# for idx, email_data in enumerate(mbox_data):
#     print('Parsing email {0} of {1}'.format(idx, num_emails))

4

In [13]:
# # Check some info
# for i in list(range(len(mbox_data))):
#     print(mbox_data[i]["To"])
    
# for i in list(range(len(mbox_data))):
#     print(mbox_data[i]["Subject"])
    
# for i in list(range(len(mbox_data))):
#     print(mbox_data[i]["From"])
    
# for i in list(range(len(mbox_data))):
#     print(mbox_data[i]["Date"])

In [14]:
body_list = []

for msg in mbox_data:
    if msg.is_multipart():    #Walk through the parts of the email to find the text body.
        for part in msg.walk():
            if part.is_multipart(): # If part is multipart, walk through the subparts.
                for subpart in part.walk():
                    if subpart.get_content_type() == 'text/plain':
                        body = subpart.get_payload(decode=False) # Get the subpart payload (i.e the message body)
                        body_list.append(body)
                    for k,v in subpart.items():
                            if k == 'Content-Transfer-Encoding':
                                cte = v             # Keep the Content Transfer Encoding
            elif subpart.get_content_type() == 'text/plain':
                body = part.get_payload(decode=False)           # part isn't multipart Get the payload
                for k,v in part.items():
                    if k == 'Content-Transfer-Encoding':
                        cte = v                      # Keep the Content Transfer Encoding

# body_list

In [15]:
body_clean = []
for i, b in enumerate(body_list):
    index = body_list[i].find('Welches')
    clean = body_list[i][index:]
    body_clean.append(clean)
# body_clean

In [33]:
question_1 = 'Welches Tesla Modell m=C3=B6chtest Du verkaufen?'
question_2 = 'Ausweisbare MWST?'
question_3 = 'Datum der Erstzulassung'
question_4 = 'Kilometerstand'
question_5 = 'Antriebsversion'
question_6 = 'Lackierungen'
question_7 = 'R=C3=A4der'
question_8 = 'Innenansicht'
question_9 = 'Autopilot'
question_10 = 'Unfallfahrzeug:'
question_11 = 'Brutto-Listenpreis'
question_12 = 'Name'
question_13 = 'Strasse und Hausnummer'
question_14 = 'PLZ und Ort'
question_15 = 'Telefonische Erreichbarkeit'
question_16 = 'E-Mail'
question_17 = 'Datenschutzerkl=C3=A4rung'
question_18 = 'Ankauf AGB'
questions = [question_1, question_2, question_3, question_4, question_5, question_6, question_7, question_8, question_9, question_10, question_11, question_12, question_13, question_14, question_15, question_16, question_17, question_18]
questions_len = []
for question in questions:
    question_len = len(question)
    questions_len.append(question_len)
# questions_len
next_question = []
for i in list(range(len(questions))):
    if i == (len(questions)-1):
        next_question.append('Gesendet von sellyourtesla.de')
    else:
        next_q = questions[(i+1)]
        next_question.append(next_q)
# next_question

In [34]:
answers_list = []

for body_n in body_clean:
    answers = []
    for i, x, s in zip(questions_len, questions, next_question):
        index = body_n.find(x)
        index_plus_one = body_n.find(s)
        answer = body_n[(index+i):(index_plus_one)]
        answers.append(answer[2:-2])
    answers_list.append(answers)
answers_list

[['Model 3',
  'Ja',
  '6363636366',
  'Ausgew=C3=A4hlter Wert: 11252 (0 min / 12000 max)',
  'Maximale Reichweite',
  'Red Multi-Coat',
  'Model 3 - 18" Aero Felgen',
  'Komplett schwarz',
  'Volles Potential f=C3=BCr autonomes Fahren',
  'Ja',
  '999999',
  'hhh ddd',
  'ddd',
  'ddd',
  '010101100101',
  'hhhh@hhhh.de',
  'Hiermit best=C3=A4tige ich, dass ich die nachfolgende Datenschutzerkl=C3=A4=\nrung\ngelesen habe und akzeptiere.\n*Ankaufs AGB*\nHiermit best=C3=A4tige ich, dass ich die nachfolgende Datenschutzerkl=C3=A4=\nrung\ngelesen habe und akzeptiere.\nGesendet von sellyourtesla.',
  'la Modell m=C3=B6chtest Du verkaufen?*\nModel 3\n*Ausweisbare MWST?*\nJa\n*Datum der Erstzulassung*\n6363636366\n*Kilometerstand*\nAusgew=C3=A4hlter Wert: 11252 (0 min / 12000 max)\n*Antriebsversion*\nMaximale Reichweite\n*Lackierungen*\nRed Multi-Coat\n*R=C3=A4der*\nModel 3 - 18" Aero Felgen\n*Innenansicht*\nKomplett schwarz\n*Autopilot*\nVolles Potential f=C3=BCr autonomes Fahren\n*Unfallfah

In [38]:
df = pd.DataFrame(answers_list)
df.columns=questions
df

,Welches Tesla Modell m=C3=B6chtest Du verkaufen?,Ausweisbare MWST?,Datum der Erstzulassung,Kilometerstand,Antriebsversion,Lackierungen,R=C3=A4der,Innenansicht,Autopilot,Unfallfahrzeug:,Brutto-Listenpreis,Name,Strasse und Hausnummer,PLZ und Ort,Telefonische Erreichbarkeit,E-Mail,Datenschutzerkl=C3=A4rung,Ankauf AGB
0,Model 3,Ja,6363636366,Ausgew=C3=A4hlter Wert: 11252 (0 min / 12000 max),Maximale Reichweite,Red Multi-Coat,"Model 3 - 18"" Aero Felgen",Komplett schwarz,Volles Potential f=C3=BCr autonomes Fahren,Ja,999999,hhh ddd,ddd,ddd,010101100101,hhhh@hhhh.de,"Hiermit best=C3=A4tige ich, dass ich die nachf...",la Modell m=C3=B6chtest Du verkaufen?*\nModel ...
1,Model 3,Ja,16.5.2021,Ausgew=C3=A4hlter Wert: 9429 (0 min / 12000 max),Maximale Reichweite,Pearl White Multi-Coat,"Model 3 - 18"" Aero Felgen",Komplett schwarz,Basis,Nein,55000,G=C3=BCnter Meisinger,94116 Hutthurm,Hutthurm,015115617854,g.meisinger@mlogica.de,"Hiermit best=C3=A4tige ich, dass ich die nachf...","Hiermit best=C3=A4tige ich, dass ich die nachf..."
2,Model 3,Ja,21.04.2020,Ausgew=C3=A4hlter Wert: 2847 (0 min / 12000 max),Standard Plus,Pearl White Multi-Coat,"Model 3 - 18"" Aero Felgen",Komplett schwarz,Basis,Ja\n*Besch=C3=A4digungen*\nTest Test\n*Zubeh=C...,Test,Test Test,Test,Test,28971309827,test@test.de,"Hiermit best=C3=A4tige ich, dass ich die nachf...","Hiermit best=C3=A4tige ich, dass ich die nachf..."
3,Model Y,Ja,09.10.2021,Ausgew=C3=A4hlter Wert: 1432 (0 min / 12000 max),Performance,Red Multi-Coat,"Model Y - 20"" Induction Felgen",Schwarz und wei=C3=9F,Enhanced\n*Name*\nJustin Breschul\n*Strasse un...,dell m=C3=B6chtest Du verkaufen?*\nModel Y\n*A...,l m=C3=B6chtest Du verkaufen?*\nModel Y\n*Ausw...,Justin Breschul,Sonderschulstrasse 1,42105 Wuppertal,0189765432,justin@gmail.com,"Hiermit best=C3=A4tige ich, dass ich die nachf...",la Modell m=C3=B6chtest Du verkaufen?*\nModel ...


In [41]:
df.to_csv(path_or_buf='../Data/parsed_emails.csv', index=False)
df.to_csv(path_or_buf='../Data/parsed_emails.csv', index=True)

In [43]:
data = df.reset_index(drop=True)
data

,Welches Tesla Modell m=C3=B6chtest Du verkaufen?,Ausweisbare MWST?,Datum der Erstzulassung,Kilometerstand,Antriebsversion,Lackierungen,R=C3=A4der,Innenansicht,Autopilot,Unfallfahrzeug:,Brutto-Listenpreis,Name,Strasse und Hausnummer,PLZ und Ort,Telefonische Erreichbarkeit,E-Mail,Datenschutzerkl=C3=A4rung,Ankauf AGB
0,Model 3,Ja,6363636366,Ausgew=C3=A4hlter Wert: 11252 (0 min / 12000 max),Maximale Reichweite,Red Multi-Coat,"Model 3 - 18"" Aero Felgen",Komplett schwarz,Volles Potential f=C3=BCr autonomes Fahren,Ja,999999,hhh ddd,ddd,ddd,010101100101,hhhh@hhhh.de,"Hiermit best=C3=A4tige ich, dass ich die nachf...",la Modell m=C3=B6chtest Du verkaufen?*\nModel ...
1,Model 3,Ja,16.5.2021,Ausgew=C3=A4hlter Wert: 9429 (0 min / 12000 max),Maximale Reichweite,Pearl White Multi-Coat,"Model 3 - 18"" Aero Felgen",Komplett schwarz,Basis,Nein,55000,G=C3=BCnter Meisinger,94116 Hutthurm,Hutthurm,015115617854,g.meisinger@mlogica.de,"Hiermit best=C3=A4tige ich, dass ich die nachf...","Hiermit best=C3=A4tige ich, dass ich die nachf..."
2,Model 3,Ja,21.04.2020,Ausgew=C3=A4hlter Wert: 2847 (0 min / 12000 max),Standard Plus,Pearl White Multi-Coat,"Model 3 - 18"" Aero Felgen",Komplett schwarz,Basis,Ja\n*Besch=C3=A4digungen*\nTest Test\n*Zubeh=C...,Test,Test Test,Test,Test,28971309827,test@test.de,"Hiermit best=C3=A4tige ich, dass ich die nachf...","Hiermit best=C3=A4tige ich, dass ich die nachf..."
3,Model Y,Ja,09.10.2021,Ausgew=C3=A4hlter Wert: 1432 (0 min / 12000 max),Performance,Red Multi-Coat,"Model Y - 20"" Induction Felgen",Schwarz und wei=C3=9F,Enhanced\n*Name*\nJustin Breschul\n*Strasse un...,dell m=C3=B6chtest Du verkaufen?*\nModel Y\n*A...,l m=C3=B6chtest Du verkaufen?*\nModel Y\n*Ausw...,Justin Breschul,Sonderschulstrasse 1,42105 Wuppertal,0189765432,justin@gmail.com,"Hiermit best=C3=A4tige ich, dass ich die nachf...",la Modell m=C3=B6chtest Du verkaufen?*\nModel ...


In [44]:
data.to_csv(path_or_buf='../Data/parsed_emails.csv', index=False)
